In [5]:
!pip install mlflow

  Using cached mlflow-1.23.1-py3-none-any.whl (15.6 MB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached GitPython-3.1.18-py3-none-any.whl (170 kB)
  Using cached protobuf-3.19.6-cp36-cp36m-win32.whl (776 kB)
  Using cached PyYAML-6.0.1-cp36-cp36m-win32.whl (137 kB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
  Using cached sqlparse-0.4.4-py3-none-any.whl (41 kB)
  Using cached prometheus_flask_exporter-0.23.0-py3-none-any.whl (18 kB)
  Using cached waitress-2.0.0-py3-none-any.whl (56 kB)
  Using cached databricks-cli-0.17.8.tar.gz (85 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached docker-5.0.3-py2.py3-none-any.whl (146 kB)
  Using cached alembic-1.7.7-py3-none-any.whl (210 kB)
  Using cached Flask-2.0.3-py3-none-any.whl (95 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
  Using cached gitdb-4.0.9-py3-none-any.whl (63 kB)
  Using cached Mako-1.1.6-py2

In [6]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, plot_roc_curve, confusion_matrix
from sklearn.model_selection import KFold
import mlflow
import mlflow.sklearn

c:\Users\1166061\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: MLflow support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use MLflow will continue to work without modification, but Python 3.6 users will no longer get access to the latest MLflow features and bugfixes. We recommend that you upgrade to Python 3.7 or newer.
  if sys.path[0] == '':


In [7]:
df = pd.read_csv('../data/creditcard.csv')
df = df.drop('Time', axis=1) # 'Time' 컬럼 삭제 : 이상 징후를 찾는데 그다지 유용하지 않고 데이터의 복잡성만 가중
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [8]:
normal = df[df.Class==0].sample(frac=0.5, random_state=2020).reset_index(drop=True)
anomaly=df[df.Class==1]

In [9]:
print(f'Normal : {normal.shape}')
print(f'Anomaly : {anomaly.shape}')

Normal : (142158, 30)
Anomaly : (492, 30)


In [10]:
normal_train, normal_test = train_test_split(normal, test_size=0.2, random_state=2020)
anomaly_train, anomaly_test = train_test_split(anomaly, test_size=0.2, random_state=2020)
normal_train, normal_validate = train_test_split(normal_train, test_size=0.25, random_state=2020)
anomaly_train, anomaly_validate =train_test_split(anomaly_train, test_size=0.25, random_state=2020)

In [11]:
x_train = pd.concat((normal_train, anomaly_train))
x_test = pd.concat((normal_test, anomaly_test))
x_validate = pd.concat((normal_validate, anomaly_validate))
y_train = np.array(x_train['Class'])
y_test = np.array(x_test['Class'])
y_validate = np.array(x_validate['Class'])
x_train = x_train.drop('Class', axis=1)
x_test = x_test.drop('Class', axis=1)
x_validate = x_validate.drop('Class', axis=1)

In [12]:
scaler = StandardScaler()
scaler.fit(pd.concat((normal, anomaly)).drop('Class', axis=1))
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_validate = scaler.transform(x_validate)

In [13]:
def train(sk_model, x_train, y_train):
    sk_model = sk_model.fit(x_train, y_train)
    train_acc = sk_model.score(x_train, y_train)
    mlflow.log_metric('train_acc', train_acc)
    print(f'Train Accuracy : {train_acc:.3%}')

In [ ]:
def evaluate(sk_model, x_test, y_test):
    eval_acc = sk_model.score(x_test, y_test)
    preds = sk_model.predict(x_test)
    auc_score = roc_auc_score(y_test, preds)
    mlflow.log_metric('eval_acc', eval_acc)
    mlflow.log_metric('auc_score', auc_score)
    print(f'Auc Score : {auc_score:.3%}')
    print(f'Eval Accuracy : {eval_acc:.3%}')
    roc_plot = plot_roc_curve(sk_model, x_test, y_test, name='scikit-learn ROC Curve')
    plt.savefig('sklearn_roc_plot.png')
    plt.show()
    plt.clf()
    conf_matrix = confusion_matrix(y_test, preds)
    